<a href="https://colab.research.google.com/github/QuyenNguyen0611/Sneaker-Data-Analysis/blob/main/sneaker_sales_seasonality_and_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load data for drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install numpy==1.24.4
!pip install pmdarima==2.0.4

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.fft import fft
from plotly.subplots import make_subplots

In [ ]:
# Load the file
file_path = '/content/drive/MyDrive/Data Analysis Projects/sneaker_dataset.csv'
data = pd.read_csv(file_path)
data.head()

,name,type,total_produced,total_sold,damaged,month,year,edition,price,gender,sell_through_rate,damage_rate,unsold_inventory,estimated_revenue,quarter,date,is_limited_edition,price_bucket,manufacturing_date,selling_date
0,Nike Air Force 1 '07 Sneakers,Creamy,24592,5819,9012,September,2017,Limited,115,Men,23.662167,36.646064,9761,669185,Q3,2017-09-01,1,High,2017-09-01,2017-11-06
1,Adidas Originals Samba OG Shoes,Creamy,81482,16395,38698,September,2019,Standard,100,Men,20.121008,47.492698,26389,1639500,Q3,2019-09-01,0,Mid,2019-09-01,2019-09-30
2,Air Jordan 1 Mid Shoes,Peanut Butter,76237,8478,26062,January,2023,Special Release,110,Women,11.120584,34.185500,41697,932580,Q1,2023-01-01,1,High,2023-01-01,2023-03-28
3,Red Tape Casual Sneakers,Brownie,46463,5425,10463,October,2022,Anniversary,35,Men,11.675957,22.518994,30575,189875,Q4,2022-10-01,0,Low,2022-10-01,2022-12-15
4,Nike Court Vision Low Shoes,Peanut Butter,54118,11698,6078,April,2019,Standard,80,Women,21.615729,11.231014,36342,935840,Q2,2019-04-01,0,Mid,2019-04-01,2019-05-06


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                500 non-null    object 
 1   type                500 non-null    object 
 2   total_produced      500 non-null    int64  
 3   total_sold          500 non-null    int64  
 4   damaged             500 non-null    int64  
 5   month               500 non-null    object 
 6   year                500 non-null    int64  
 7   edition             500 non-null    object 
 8   price               500 non-null    int64  
 9   gender              500 non-null    object 
 10  sell_through_rate   500 non-null    float64
 11  damage_rate         500 non-null    float64
 12  unsold_inventory    500 non-null    int64  
 13  estimated_revenue   500 non-null    int64  
 14  quarter             500 non-null    object 
 15  date                500 non-null    object 
 16  is_limit

The dataset contains 500 rows and 20 columns. Upon review, several columns appear redundant and can be removed to streamline the analysis.Notably, the current damaged and unsold rate values are in percentage format (e.g., 40, 50), so I plan to compute the damaged rate, sold rate, and unsold rate decimal format (e.g., 0.4, 0.5) for consistency and easier comparison.

# Data Cleaning

In [ ]:
# Remove redundancy columns month, year, quarter, date
data.drop(['month', 'year', 'quarter', 'date','sell_through_rate', 'damage_rate'], axis=1, inplace=True)

# Convert manfacturing_date and selling_date into datatime type
data['manufacturing_date'] = pd.to_datetime(data['manufacturing_date'])
data['selling_date'] = pd.to_datetime(data['selling_date'])

# Calculate damaged_rate, sold_rate, unsold_rate
data['damaged_rate'] = (data['damaged'] / data['total_produced'])
data['sold_rate'] = (data['total_sold'] / data['total_produced'])
data['unsold_rate'] = (data['unsold_inventory'] / data['total_produced'])

# Days between 'manufaturing_date' and 'selling_date' based on 'name' of sneaker
data['days'] = (data['selling_date'] - data['manufacturing_date']).dt.days

# Drop data if year = 2025 and 2015
data = data[data['selling_date'].dt.year != 2025]
data = data[data['selling_date'].dt.year != 2015]
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 459 entries, 0 to 497
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   name                459 non-null    object        
 1   type                459 non-null    object        
 2   total_produced      459 non-null    int64         
 3   total_sold          459 non-null    int64         
 4   damaged             459 non-null    int64         
 5   edition             459 non-null    object        
 6   price               459 non-null    int64         
 7   gender              459 non-null    object        
 8   unsold_inventory    459 non-null    int64         
 9   estimated_revenue   459 non-null    int64         
 10  is_limited_edition  459 non-null    int64         
 11  price_bucket        459 non-null    object        
 12  manufacturing_date  459 non-null    datetime64[ns]
 13  selling_date        459 non-null    datetime64[ns]
 14 

In [ ]:
data.head()

,name,type,total_produced,total_sold,damaged,edition,price,gender,unsold_inventory,estimated_revenue,is_limited_edition,price_bucket,manufacturing_date,selling_date,damaged_rate,sold_rate,unsold_rate,days
0,Nike Air Force 1 '07 Sneakers,Creamy,24592,5819,9012,Limited,115,Men,9761,669185,1,High,2017-09-01,2017-11-06,0.366461,0.236622,0.396918,66
1,Adidas Originals Samba OG Shoes,Creamy,81482,16395,38698,Standard,100,Men,26389,1639500,0,Mid,2019-09-01,2019-09-30,0.474927,0.201210,0.323863,29
2,Air Jordan 1 Mid Shoes,Peanut Butter,76237,8478,26062,Special Release,110,Women,41697,932580,1,High,2023-01-01,2023-03-28,0.341855,0.111206,0.546939,86
3,Red Tape Casual Sneakers,Brownie,46463,5425,10463,Anniversary,35,Men,30575,189875,0,Low,2022-10-01,2022-12-15,0.225190,0.116760,0.658050,75
4,Nike Court Vision Low Shoes,Peanut Butter,54118,11698,6078,Standard,80,Women,36342,935840,0,Mid,2019-04-01,2019-05-06,0.112310,0.216157,0.671533,35


In [ ]:
#Save data to csv file and download
#data.to_csv('sneaker_dataset_cleaned.csv', index=False)
#from google.colab import files
#files.download('sneaker_dataset_cleaned.csv')

# Data Analysis
1. Trend of estimated revenue over time
2. The dataset have seasonality or not? Find average seasonal impact by month
3. Sneaker sales forecast for the next 12 months and peak season marks

### Estimated Revenue over time

In [ ]:
# Calculate total estimated revenue each year
total_estimated_revenue = data.groupby(data['selling_date'].dt.year)['estimated_revenue'].sum()
total_estimated_revenue

,estimated_revenue
selling_date,
2016,142185960
2017,114371745
2018,125698545
2019,117702265
2020,142689220
2021,123641300
2022,147393710
2023,112194945
2024,108855795


In [ ]:
# Plot line chart for total_estimated_revenue
fig = go.Figure()
fig.add_trace(go.Scatter(x=total_estimated_revenue.index, y=total_estimated_revenue.values, mode='lines+markers', name='Total Estimated Revenue Over Time'))

The total estimated revenue has shown significant annual fluctuations between 2016 and 2024, peaking notably in 2022. However, following this peak, the chart indicates a consistent decline in estimated revenue from 2022 through 2024, reaching the lowest point in the displayed period.

## Seasonal impact by month

In [ ]:
monthly_sales = (
    data.groupby(data['selling_date'].dt.to_period('M'))['estimated_revenue']
    .sum()
    .to_timestamp()
)
monthly_sales

,estimated_revenue
selling_date,
2016-01-01,20307435
2016-02-01,3694220
2016-03-01,9224240
2016-04-01,19622345
2016-05-01,19365170
...,...
2024-08-01,13432210
2024-09-01,19663380
2024-10-01,2601300


In [ ]:
# Reset index to turn datetime index into a regular column
monthly_sales_df = monthly_sales.reset_index()
monthly_sales_df.rename(columns={'selling_date': 'date', 'estimated_revenue': 'sales'}, inplace=True)

# Extract month from date for color mapping
monthly_sales_df['month'] = monthly_sales_df['date'].dt.month
colors = {
    1: '#AEC6CF',
    2: '#FFB347',
    3: '#77DD77',
    4: '#FFD1DC',
    5: '#FDFD96',
    6: '#CBAACB',
    7: '#B39EB5',
    8: '#FFDAC1',
    9: '#E0BBE4',
    10: '#D5AAFF',
    11: '#B5EAD7',
    12: '#FF9AA2'
}

# Map color for each bar based on month
monthly_sales_df['color'] = monthly_sales_df['month'].map(colors)

# Create Plotly bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly_sales_df['date'],
    y=monthly_sales_df['sales'],
    marker_color=monthly_sales_df['color'],  # Color bars by month
    name='Monthly Sales'
))

# Update chart layout
fig.update_layout(title='Monthly Sales',
                  yaxis_title='Estimated Revenue')

# Show interactive chart
fig.show()

An analysis of the sneaker revenue chart from 2016 to 2024 reveals a highly volatile sales pattern without a clear long-term upward or downward trend. The significant monthly fluctuations suggest that revenue is heavily influenced by short-term factors like promotional campaigns, new product launches, and external market events. While the overall historical data appears erratic, with major spikes in Feb-2018 and Mar-Apr 2020, a new seasonal trend seems to be forming. Notably, for the last three years, from 2022 to 2024, September has consistently marked the annual revenue peak, indicating an emerging and predictable high-sales period.

The preceding observations are based on visual analysis. To obtain an objective and quantitative assessment, the next step is to apply Time Series Decomposition. This method will clearly separate the Trend, Seasonality, and Residual components of the data.

In [ ]:
# Perform seasonal decomposition
decomposition = seasonal_decompose(monthly_sales, model='additive', period=12)

# Create subplots with 4 rows to display each component separately
fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                    vertical_spacing=0.05,
                    subplot_titles=('Observed', 'Trend', 'Seasonal', 'Residual'))

# Plot Observed component
fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.observed,
                         mode='lines', name='Observed', line=dict(color='blue')),
              row=1, col=1)

# Plot Trend component
fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.trend,
                         mode='lines', name='Trend', line=dict(color='orange')),
              row=2, col=1)

# Plot Seasonal component
fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.seasonal,
                         mode='lines', name='Seasonal', line=dict(color='green')),
              row=3, col=1)

# Plot Residual component as scatter points
fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.resid,
                         mode='markers', name='Residual', marker=dict(color='red')),
              row=4, col=1)

# Update overall layout
fig.update_layout(height=1000, width=1000,
                  title_text="Seasonal Decomposition of Monthly Sales (Plotly)",
                  showlegend=False)

# Show interactive chart
fig.show()


The Trend component shows noticeable fluctuations in the overall sales level over time. Peaks and troughs are clearly visible, indicating that, beyond short-term seasonal effects, the long-term sales trajectory experiences significant ups and downs. These fluctuations may be associated with external factors such as product launches, promotions, or market changes. Despite some recovery phases, there is no clear, consistent long-term upward or downward trend, suggesting that sales dynamics are highly event-driven rather than following a stable growth pattern.

The Seasonal component chart clearly shows a seasonal trend that repeats itself regularly over a 12-month cycle. The peaks and troughs repeat relatively consistently over the years, confirming that sneaker sales data has a clear and consistent seasonal element over the years.

The residual component appears randomly scattered around zero, with no obvious pattern or structure over time. This indicates that the decomposition model has successfully extracted the main trend and seasonal effects, leaving behind random fluctuations that likely represent irregular events, outliers, or noise in the data. Some large positive and negative spikes are visible, suggesting occasional extreme deviations, which could be due to unexpected events such as promotions, market disruptions, or external shocks. Overall, the residual pattern suggests that the model adequately captures the systematic structure in the data.

In [ ]:
# Create 'month' from seasonal component
seasonal_component = decomposition.seasonal
seasonal_df = seasonal_component.reset_index()
seasonal_df['month'] = seasonal_df['selling_date'].dt.month

# Group by month and calculate average value
monthly_seasonality = seasonal_df.groupby('month')[seasonal_component.name].mean()

# Create Plotly chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly_seasonality.index,
    y=monthly_seasonality.values,
    marker_color='lightblue',
    name='Seasonal Impact'
))

# Customize layout
fig.update_layout(
    title='Average Seasonal Impact by Month',
    xaxis_title='Month',
    yaxis_title='Seasonal Impact',
    xaxis=dict(tickmode='linear'),
    yaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor='black')
)

fig.show()


The bar chart illustrates the average seasonal effect on revenue for each month. Positive values indicate months where seasonal factors contribute to higher-than-average sales, while negative values indicate months with a downward seasonal influence.

* Months 2, 3, 4, 8, 9 and 12 exhibit a positive seasonal impact, suggesting these are typically strong sales months due to recurring seasonal patterns.

* Months 1, 5, 6, 7, 10, and 11 show a negative seasonal effect, indicating these periods are naturally weaker for sales.

* The most significant positive seasonal impact occurs in December, likely due to end-of-year shopping behavior.

* The most pronounced negative seasonal impact appears in January and June, potentially reflecting post-holiday slowdowns or off-peak buying periods.



## Predict revenue next year

In [ ]:
# Use auto_arima to choose best model
stepwise_model = auto_arima(monthly_sales,
                            start_p=0, start_q=0,
                            max_p=3, max_q=3,
                            seasonal=True, m=12,  # m=12 means 12 months
                            start_P=0, start_Q=0,
                            max_P=2, max_Q=2,
                            d=1, D=1,
                            trace=True,
                            error_action='ignore',
                            suppress_warnings=True,
                            stepwise=True)

# Summary the best model
print(stepwise_model.summary())

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=3365.505, Time=0.06 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=3328.404, Time=0.33 sec


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



 ARIMA(0,1,1)(0,1,1)[12]             : AIC=3288.761, Time=0.41 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=3313.184, Time=0.12 sec


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



...

 ARIMA(0,1,3)(1,1,2)[12]             : AIC=3290.898, Time=1.52 sec


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



 ARIMA(1,1,3)(0,1,1)[12]             : AIC=3288.373, Time=1.03 sec


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



 ARIMA(0,1,3)(0,1,1)[12] intercept   : AIC=3295.555, Time=3.52 sec

Best model:  ARIMA(0,1,3)(0,1,1)[12]          
Total fit time: 20.685 seconds
                                      SARIMAX Results                                       
Dep. Variable:                                    y   No. Observations:                  108
Model:             SARIMAX(0, 1, 3)x(0, 1, [1], 12)   Log Likelihood               -1638.756
Date:                              Sat, 28 Jun 2025   AIC                           3287.512
Time:                                      21:08:37   BIC                           3300.281
Sample:                                  01-01-2016   HQIC                          3292.672
                                       - 12-01-2024                                         
Covariance Type:                                opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------

In [ ]:
# Fit SARIMAX model with the best parameters
model = SARIMAX(monthly_sales,
                order=(0,1,3),
                seasonal_order=(0,1,1,12),
                enforce_stationarity=False,
                enforce_invertibility=False)

results = model.fit()

# Print model summary
print(results.summary())

# Forecast the next 12 months
forecast = results.get_forecast(steps=12)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

# Create Plotly figure
fig = go.Figure()

# Plot actual sales
fig.add_trace(go.Scatter(
    x=monthly_sales.index,
    y=monthly_sales.values,
    mode='lines',
    name='Actual Sales',
    line=dict(color='blue')
))

# Plot forecasted sales
fig.add_trace(go.Scatter(
    x=forecast_mean.index,
    y=forecast_mean.values,
    mode='lines+markers',
    name='Forecast',
    line=dict(color='red')
))

# Plot confidence interval as shaded area
fig.add_trace(go.Scatter(
    x=pd.concat([forecast_ci.index.to_series(), forecast_ci.index.to_series()[::-1]]),
    y=pd.concat([forecast_ci.iloc[:, 1], forecast_ci.iloc[:, 0][::-1]]),
    fill='toself',
    fillcolor='rgba(255, 0, 0, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    showlegend=True,
    name='95% Confidence Interval'
))

# Update layout
fig.update_layout(
    title='Sneaker Sales Forecast with SARIMA (Plotly)',
    xaxis_title='Time',
    yaxis_title='Estimated Revenue',
    hovermode='x unified'
)

fig.show()


                                      SARIMAX Results                                       
Dep. Variable:                    estimated_revenue   No. Observations:                  108
Model:             SARIMAX(0, 1, 3)x(0, 1, [1], 12)   Log Likelihood               -1360.848
Date:                              Sat, 28 Jun 2025   AIC                           2731.695
Time:                                      21:06:09   BIC                           2743.542
Sample:                                  01-01-2016   HQIC                          2736.441
                                       - 12-01-2024                                         
Covariance Type:                                opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.7901      0.247     -3.202      0.001      -1.274      -0.306
ma.L2         -0.18

Based on the SARIMA forecast results:

* Sneaker sales are expected to fluctuate around the historical average over the next 12 months.
* Significant uncertainty remains, reflecting potential market volatility.

**Recommended actions:**

* Closely monitor actual monthly sales and adjust procurement plans accordingly.
* Focus promotional and marketing activities on the months identified as high season from previous analysis.
* Do not rely solely on statistical models; integrate market insights and upcoming sales campaigns to refine forecasts.

